In [1]:
from constraint import Problem
import random
from PyQt5.QtCore import Qt, QTimer
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QComboBox, QPushButton, QGridLayout, QVBoxLayout, QTableWidget, QTableWidgetItem, QHBoxLayout, QHeaderView, QMessageBox
import sys

def generate_variable_names(variables, suffixes):
    new_variables = []
    for var in variables:
        for suffix in suffixes:
            new_var = var + suffix
            new_variables.append(new_var)
    return new_variables

class TT:
    def __init__(self, v, l, d, dl):
        self.problem = Problem()
        self.variables = v
        self.domain = d
        self.labs = l
        self.domainL = dl
        self._setup_problem()

    def _setup_problem(self):
        # Add domain to theory and add basic constraints
        for v in self.variables:
            self.problem.addVariables(v, self.domain)
            self.individual_class_constraints(v)

        # Avoid room clashes among different classes
        v1, v2, v3, v4 = self.variables
        for a in v1:
            for b in v2:
                self.problem.addConstraint(self.different_room, (a,b))
            for c in v3:
                self.problem.addConstraint(self.different_room, (a,c))
            for d in v4:
                self.problem.addConstraint(self.different_room, (a,d))

        for b in v2:
            for c in v3:
                self.problem.addConstraint(self.different_room, (b,c))
            for d in v4:
                self.problem.addConstraint(self.different_room, (b,d))

        for c in v3:
            for d in v4:
                self.problem.addConstraint(self.different_room, (c,d))
                
        # This portion is for lab
        
        # Add lab domain to lab variables
        self.problem.addVariable('Linear Circuit Analysis - LABa', [(day, slot, 'Circuit Analysis Lab (Z-122)') for day in days for slot in slots])
        self.problem.addVariable('Linear Circuit Analysis - LABb', [(day, slot, 'Circuit Analysis Lab (Z-122)') for day in days for slot in slots])
        self.problem.addVariable('Linear Circuit Analysis - LABc', [(day, slot, 'Circuit Analysis Lab (Z-122)') for day in days for slot in slots])
        self.problem.addVariable('Linear Circuit Analysis - LABd', [(day, slot, 'Circuit Analysis Lab (Z-122)') for day in days for slot in slots])
        self.problem.addVariable('Embedded Systems Workshop - LABa', [(day, slot, 'VLSI LAB') for day in days for slot in slots])
        self.problem.addVariable('Embedded Systems Workshop - LABb', [(day, slot, 'VLSI LAB') for day in days for slot in slots])
        self.problem.addVariable('Electric Circuit Analysis II - LABa', [(day, slot, 'Circuit Analysis Lab (Z-122)') for day in days for slot in slots])
        self.problem.addVariable('Electric Circuit Analysis II - LABb', [(day, slot, 'Circuit Analysis Lab (Z-122)') for day in days for slot in slots])
        self.problem.addVariable('Introduction to Chemistry - LABa', [(day, slot, 'Applied&Analytical Chem Lab (I-106)') for day in days for slot in slots])
        self.problem.addVariable('Introduction to Chemistry - LABb', [(day, slot, 'Applied&Analytical Chem Lab (I-106)') for day in days for slot in slots])
        self.problem.addVariable('Applied Physics - LABa', [(day, slot, 'Physics Lab (Z-119)') for day in days for slot in slots])
        self.problem.addVariable('Applied Physics - LABb', [(day, slot, 'Physics Lab (Z-119)') for day in days for slot in slots])
        added = ['Linear Circuit Analysis - LABa', 'Linear Circuit Analysis - LABb', 'Linear Circuit Analysis - LABc', 'Linear Circuit Analysis - LABd',
                 'Embedded Systems Workshop - LABa', 'Embedded Systems Workshop - LABb', 'Electric Circuit Analysis II - LABa',
                 'Electric Circuit Analysis II - LABb','Introduction to Chemistry - LABa', 'Introduction to Chemistry - LABb',
                 'Applied Physics - LABa', 'Applied Physics - LABb']
        for labs in self.labs:
            for lab in labs:
                if lab not in added:
                    self.problem.addVariable(lab, self.domainL)
        l1, l2, l3, l4 = self.labs
        
        # Avoid clash with theory e.g. BCE1 theory and BCE1 lab
        for l in l1:
            for v in v1:
                self.problem.addConstraint(self.clash_avoidance, (l, v))
                if l[:-7] == v[:-1]:
                    self.problem.addConstraint(self.different_day, (l, v))
        for l in l2:
            for v in v2:
                self.problem.addConstraint(self.clash_avoidance, (l, v))
                if l[:-7] == v[:-1]:
                    self.problem.addConstraint(self.different_day, (l, v))
        for l in l3:
            for v in v3:
                self.problem.addConstraint(self.clash_avoidance, (l, v))
                if l[:-7] == v[:-1]:
                    self.problem.addConstraint(self.different_day, (l, v))
        for l in l4:
            for v in v4:
                self.problem.addConstraint(self.clash_avoidance, (l, v))
                if l[:-7] == v[:-1]:
                    self.problem.addConstraint(self.different_day, (l, v))

        # Avoid day slot clashes e.g. BCE1 lab1 and BCE1 lab2
        for lab in self.labs:
            for i in range(len(lab)):
                for j in range(i + 1, len(lab)):
                    prefix_i = lab[i][:-7]  # Extract prefix of the first element
                    prefix_j = lab[j][:-7]  # Extract prefix of the second element
                    if prefix_i != prefix_j:  # Check if prefixes are different
                        self.problem.addConstraint(self.clash_avoidance, (lab[i], lab[j]))

            # Add lab slots adjacently i.e. both lab slots of lab1 should be adjacent
            m = 0
            while m < len(lab):
                lectures = 2
                while lectures > 1:
                    for n in range(m + 1, m + lectures):
                        self.problem.addConstraint(self.adjacent_lab_slots, (lab[m],lab[n]))
                    m += 1
                    lectures -= 1
                m += 1

        # Different rooms for labs e.g. BCE1 lab and BCE3 lab
        for a in l1:
            for b in l2:
                self.problem.addConstraint(self.different_room, (a,b))
            for c in l3:
                self.problem.addConstraint(self.different_room, (a,c))
            for d in l4:
                self.problem.addConstraint(self.different_room, (a,d))

        for b in l2:
            for c in l3:
                self.problem.addConstraint(self.different_room, (b,c))
            for d in l4:
                self.problem.addConstraint(self.different_room, (b,d))

        for c in l3:
            for d in l4:
                self.problem.addConstraint(self.different_room, (c,d))
        
        
    def individual_class_constraints(self, variables):     
        for i in range(len(variables)):
            for j in range(i + 1, len(variables)):
                self.problem.addConstraint(self.clash_avoidance, (variables[i], variables[j]))
        i = 0
        while i < len(variables):
            lectures = len(suffixes)
            while lectures > 1:
                for j in range(i + 1, i + lectures):
                    self.problem.addConstraint(self.different_day, (variables[i], variables[j]))
                i += 1
                lectures -= 1
            i += 1
            
    def adjacent_lab_slots(self, p1, p2):
        return p1[0] == p2[0] and abs(int(p1[1][-1]) - int(p2[1][-1])) == 1 and p1[2] == p2[2]
        
    def clash_avoidance(self, p1, p2):
        return p1[0] != p2[0] or p1[1] != p2[1]
        
    def different_day(self, instance1, instance2):
        return instance1[0] != instance2[0]
        
    def different_room(self, instance1, instance2):
        return instance1 != instance2

    def r_solution(self):
        return self.problem.getSolution()

#  Various factors for domain
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri']
slots = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6']
rooms = ['Z-{}'.format(num) for num in range(101, 105)]
roomsL = ['COMPUTER LAB {}'.format(num) for num in range(1, 3)]

# Create theory and lab variables for different classes
v1 = ['Applied Physics', 'Linear Circuit Analysis', 'Functional English', 'Calculus and Analytic Geometry', 'Introduction to Chemistry']
v2 = ['Electric Circuit Analysis II', 'Object-Oriented Programming', 'Engineering Professionalism', 'Differential Equations', 'Discrete Mathematics']
v3 = ['Artificial Intelligence', 'Operating Systems', 'Database Systems I', 'Engineering Economics', 'Computer Organization and Architecture']
v4 = ['Digital System Design', 'Web Technologies', 'Digital Image Processing', 'Project Planning and Management']
l1 = ['Applied Physics - LAB', 'Linear Circuit Analysis - LAB', 'Introduction to Chemistry - LAB']
l2 = ['Computer-Aided Engineering Design - LAB', 'Object-Oriented Programming - LAB', 'Electric Circuit Analysis II - LAB']
l3 = ['Computer Organization and Architecture - LAB', 'Artificial Intelligence - LAB', 'Operating Systems - LAB', 'Database Systems I - LAB']
l4 = ['Digital System Design - LAB', 'Embedded Systems Workshop - LAB', 'Web Technologies - LAB', 'Digital Image Processing - LAB']

# Represent no of classes with suffixes
suffixes = ['a', 'b']

# Generate new variables with no of classes
v1 = generate_variable_names(v1, suffixes)
v2 = generate_variable_names(v2, suffixes)
v3 = generate_variable_names(v3, suffixes)
v4 = generate_variable_names(v4, suffixes)
l1 = generate_variable_names(l1, suffixes)
l2 = generate_variable_names(l2, suffixes)
l3 = generate_variable_names(l3, suffixes)
l4 = generate_variable_names(l4, suffixes)

v2.append('Linear Circuit Analysisc')
v2.append('Linear Circuit Analysisd')
l2.append('Linear Circuit Analysis - LABc')
l2.append('Linear Circuit Analysis - LABd')

# Group different classes together
variables = [v1, v2, v3, v4]
lab = [l1, l2, l3, l4]

# Create domains with factors mentioned above
domain = [(day, slot, room) for day in days for slot in slots for room in rooms]
domainL = [(day, slot, room) for day in days for slot in slots for room in roomsL]

# Create class object and get solution
solver = TT(variables, lab, domain, domainL)
solution = solver.r_solution()
#print(solution)

# Extract info from solution for each class
class_schedules = {f"BCE{i}": {variable: [] for variable in vars_labs} for i, vars_labs in enumerate(variables, start=1)}

for i, vars_labs in enumerate(variables, start=1):
    for variable in vars_labs:
        class_schedules[f"BCE{i}"][variable].append(solution[variable])

def generate_random_domains():
    domain = [(day, slot, room) for day in days for slot in slots for room in rooms]
    domainL = [(day, slot, room) for day in days for slot in slots for room in roomsL]

    list.reverse(domain)
    random.shuffle(domain)
    list.reverse(domainL)
    random.shuffle(domainL)

    return domain, domainL

# Graphics
class ScheduleApp(QWidget):
    def __init__(self):
        super().__init__()

        self.schedule = class_schedules
        self.setWindowTitle("Class Schedule App")
        self.setGeometry(100, 100, 1920, 1000)  # Set the initial size of the window
        self.setStyleSheet("background-color: #CAF1DE;")  # Set the background color

        # Create widgets
        self.class_label = QLabel("Select Class To See Schedule:")
        self.class_dropdown = QComboBox()
        self.class_dropdown.addItems(["BCE1", "BCE3", "BCE5", "BCE7"])

        self.show_schedule_button = QPushButton("Show Schedule")
        self.show_schedule_button.clicked.connect(self.show_schedule)

        self.generate_button = QPushButton("Generate")
        self.generate_button.clicked.connect(self.generate_schedules)  # Change to call the new method
        
        self.schedule_table = QTableWidget()
        self.schedule_table.setColumnCount(len(slots) + 1)  # Add one column for "Days"
        self.schedule_table.setHorizontalHeaderLabels(["Days"] + slots)  # Add "Days" column header
        
        #Set row height for the header row (column names)
        header_row_height = 73  # Adjust the height as needed
        self.schedule_table.horizontalHeader().setMinimumHeight(header_row_height)

        # Create layout
        layout = QGridLayout()
        layout.addWidget(self.class_label, 0, 0)
        layout.addWidget(self.class_dropdown, 0, 1)
        layout.addWidget(self.show_schedule_button, 0, 2)
        layout.addWidget(self.generate_button, 0, 3)
        layout.addWidget(self.schedule_table, 1, 0, 1, 4)

        self.setLayout(layout)

        # Center the window on the screen
        self.center()

        # Additional setup for the schedule table
        self.schedule_table.verticalHeader().setVisible(False)  # Hide the vertical header (row numbers)
        
        # Set column width for "Days" column
        column_width_days = 269 # Change this value as needed
        self.schedule_table.setColumnWidth(0, column_width_days)

        # Set column width for other columns
        for j in range(1, len(slots) + 1):
            column_width = 269 # Change this value as needed
            self.schedule_table.setColumnWidth(j, column_width)

        # Initialize the domains
        self.domain, self.domainL = generate_random_domains()
            
    def center(self):
        frame_geometry = self.frameGeometry()
        screen_center = QApplication.desktop().screenGeometry().center()
        frame_geometry.moveCenter(screen_center)
        self.move(frame_geometry.topLeft())

    def generate_schedules(self):
        # Reset row and column count before adding new data
        self.schedule_table.setRowCount(0)
        self.schedule_table.setColumnCount(len(slots) + 1)
        self.schedule.clear()

        # Update the instance variable self.class_schedules
        for class_name, class_vars_labs in zip(["BCE1", "BCE3", "BCE5", "BCE7"], [v1 + l1, v2 + l2, v3 + l3, v4 + l4]):
            schedule = {day: {slot: "" for slot in slots} for day in days}

            # Create a new class object and get a new solution
            new_sol = TT(variables, lab, self.domain, self.domainL)
            new_solution = new_sol.r_solution()

            for variable in class_vars_labs:
                day, slot, room = new_solution[variable]
                schedule[day][slot] = f"{variable[:-1]}\n({room})"

            # Update the instance variable self.schedule
            self.schedule[class_name] = schedule

        # Update the domains for the next click
        self.domain, self.domainL = generate_random_domains()

        
    def show_schedule(self):
        selected_class = self.class_dropdown.currentText()
        schedule = self.schedule[selected_class]
        self.display_schedule_table(schedule)

    def display_schedule_table(self, schedule):
        self.schedule_table.setRowCount(len(days))

        # Initialize the "Days" column in the schedule table
        for i, day in enumerate(days):
            item = QTableWidgetItem(day)
            item.setTextAlignment(Qt.AlignCenter)  # Set text alignment
            self.schedule_table.setItem(i, 0, item)
    
        for i, day in enumerate(days):
            for j, slot in enumerate(slots):
                item = QTableWidgetItem(schedule[day][slot])
                item.setTextAlignment(Qt.AlignCenter)  # Set text alignment
                self.schedule_table.setItem(i, j + 1, item)
                
        # Fix the typo in the following line
        for i in range(self.schedule_table.rowCount()):
            self.schedule_table.setRowHeight(i, 155)  # Adjust the row height as needed
        

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = ScheduleApp()
    window.show()
    sys.exit(app.exec_())

SystemExit: 0

C:\Users\Hasnain Hafeez\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
